In [ ]:
import sys
import argparse

import torch
import pytorch_lightning as pl

sys.path.append('../')
from model.cnf import CNF, AugmentedCNF

%load_ext autoreload
%autoreload 2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 2D-CNF

In [ ]:
# Initialize source and target
args = argparse.Namespace(num_hidden_units = 16, 
                          num_timesteps = 25,
                          source = 'gaussian',
                          target = 'gmm',
                          target_nmode = 2)       

# Initialize flow
cnf = CNF(args)

# Train
trainer = pl.Trainer(min_epochs=1, max_epochs=1)
trainer.fit(cnf)

In [ ]:
# Alternatively, load a flow
cnf = CNF.load_from_checkpoint('../workdir/time_cnf_bimodal0.ckpt')
cnf.eval()

In [ ]:
cnf.data_loss(N=10000)

In [ ]:
cnf.graph_marginals(num_marginals=10)

In [ ]:
cnf.graph_backwards_marginals(num_marginals=10)

In [ ]:
cnf.reverse_kl(num_timesteps=60, N=100000)

## Phase Space CNF

In [ ]:
# Alternatively, load a trained model
cnf = AugmentedCNF.load_from_checkpoint('../workdir/overhaul_bimodal/epoch=17-val_loss=5.11.ckpt')

In [ ]:
cnf.graph_marginals(num_marginals=10, N=100000)

In [ ]:
cnf.graph_backwards_marginals(num_marginals=10, N=100000)

In [ ]:
cnf.reverse_kl()

In [ ]:
wrapper = cnf.export_to_wrapper()

In [ ]:
wrapper.graph_end_marginals(n_samples=100000, n_integrator_steps=5)

In [ ]:
wrapper.graph_forward_marginals(n_samples=100000, n_marginals=5, n_integrator_steps=60)

In [ ]:
wrapper.graph_backward_marginals(n_samples=100000, n_marginals=5, n_integrator_steps=60)

In [ ]:
wrapper._target.q_dist.graph(100000)

In [ ]:
wrapper.graph_flow_marginals(n_marginals=5)

In [ ]:
wrapper.reverse_kl(N=10000)